# Evaluate a training dataset

## Contents of this notebook:
- 

## Imports and dataset parameters

In [37]:
import numpy as np
import rioxarray

In [4]:
dataset_dir = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/dataset_processing/baker_csm/baker_csm_stack"

In [11]:
%cp /mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/mosaic/mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif $dataset_dir

In [12]:
%ls $dataset_dir

 final_ortho_left_1.0m_holes_filled.tif
 final_ortho_left_1.0m.tif
 final_ortho_right_1.0m_holes_filled.tif
 final_ortho_right_1.0m.tif
'mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif'
 try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM_holes_filled.tif
 try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif
 try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr_holes_filled.tif
 try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr.tif


In [13]:
%cd $dataset_dir

/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/dataset_processing/baker_csm/baker_csm_stack


In [45]:
input_dem_fn = "try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif"

In [46]:
ortho1_fn = "final_ortho_left_1.0m.tif"
ortho2_fn = "final_ortho_right_1.0m.tif"

In [47]:
trierror_fn = "try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr.tif"

In [48]:
lidar_dem_fn = "mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif"

In [49]:
landcover_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/merge_pc_inference_files/ESA_WorldCover_10m_2021_v200_N48W123_Map.tif"

## For each layer, calculate mean & std to use for normalization

In [50]:
def compute_stats(layer):
    mean = layer.mean().item()
    std = layer.std().item()
    count_valid_pixels = np.count_nonzero(~np.isnan(layer))
    num_pixels = layer.size
    percent_valid = 100 * count_valid_pixels / num_pixels
    return dict(
        mean=mean,
        std=std,
        percent_valid=percent_valid
    )

In [51]:
for layer_fn in [input_dem_fn, ortho1_fn, ortho2_fn, trierror_fn, lidar_dem_fn]:
    print(f"Calculating stats for {layer_fn}")
    layer = rioxarray.open_rasterio(layer_fn, masked=True)
    stats = compute_stats(layer)
    mean = stats["mean"]
    std = stats["std"]
    percent_valid = stats["percent_valid"]
    print(f"Layer mean = {mean}, std = {std}, percent valid = {percent_valid:.2f}%")

Calculating stats for try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif
Layer mean = 1780.005126953125, std = 440.2155456542969, percent valid = 82.91%
Calculating stats for final_ortho_left_1.0m.tif
Layer mean = 463.2972106933594, std = 429.40875244140625, percent valid = 74.50%
Calculating stats for final_ortho_right_1.0m.tif
Layer mean = 412.00994873046875, std = 433.59356689453125, percent valid = 74.50%
Calculating stats for try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-IntersectionErr.tif
Layer mean = 0.15542659163475037, std = 0.16889117658138275, percent valid = 82.91%
Calculating stats for mosaic_full128_USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3_1.0m-DEM_holes_filled.tif
Layer mean = 1723.7064208984375, std = 459.1587219238281, percent valid = 100.00%


### Recalculate with extent limited to the regions used in training
- Provide a different set of rasters that only include 

In [ ]:
refined_dem = "

## Evaluate an inference output against the input DEM. What changed?

In [23]:
dem = rioxarray.open_rasterio(input_dem, masked=True)